<a href="https://colab.research.google.com/github/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# UC2 REST Tutorial

Here we are going to teach you how to interact with the UC2 microcontroller and how you can add additional functionalities. 

In order to use the client in your python environment you need the following packages to be installed:

(use the `!` to install it from within this jupyter notebook)
```py
!pip install UC2-REST
```

This code has been tested with the ESP32 WEMOS D1 R32 + CNC shield v3, where 3 stepper are connected to the board and an LED Matrix (WS2812, adafruit) is connected to the FEED pin. 

If you find an error, please report it [here](https://github.com/openUC2/UC2-REST/issues/new) 

In [ ]:
!pip install UC2-REST

## Organize all imports

First of all we need to import the `ESP32Client`. Since it is not yet a standalone pip package, we have to do that via a relaitve import, meaning that the file is in the same folder as this Jupyter Notebook

In [10]:
%load_ext autoreload 
%autoreload 2
import uc2rest
import time
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Connecting to the ESP32 via USB

Now we want to initiliaze the USB-serial connection. Therefore, connect the ESP32 to your computer using a data (!) USB cable and establish the connection. You can leave the port as "unknown" as the portfinder may identify the ESP.

**Important:** Close all applications that may be connected to the ESP (e.g. Arduino Serial Plotter)

**IMPORTANT:** Install the USB serial driver for the ESP32: https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all

In [2]:
ESP32 = uc2rest.UC2Client(serialport="unknown")

2022-10-22 23:39:46 DEBUG [UC2Client] /dev/cu.MALS
2022-10-22 23:39:46 DEBUG [UC2Client] /dev/cu.SOC
2022-10-22 23:39:46 DEBUG [UC2Client] /dev/cu.Bluetooth-Incoming-Port
2022-10-22 23:39:46 DEBUG [UC2Client] /dev/cu.MiTrueWirelessEBsBasic2
2022-10-22 23:39:46 DEBUG [UC2Client] /dev/cu.SLAB_USBtoUART
2022-10-22 23:39:49 DEBUG [UC2Client] We are connected: True on port: /dev/cu.SLAB_USBtoUART


# Moving the motor 

The following code snippets will help you moving the motors (XYZ) continously or at a known number of `steps` at a certain `speed` level (all measured in steps/s). 

The additional attributs 
- `is_blocking` states if the action is performed in the background or not; if `False` no return message will be provided
- `is_absolute` says if we go relative or absolute steps 
- `is_enabled` says if we want to "unpower" the motors once we are done (prevent overheating)

In [4]:
# move and measure
print("Current position: "+ str(ESP32.motor.get_position(axis=1)))
ESP32.motor.move_x(steps=1000, speed=1000, is_blocking=True, is_absolute=True, is_enabled=True)
print("Current position: "+ str(ESP32.motor.get_position(axis=1)))
ESP32.motor.move_x(steps=0, speed=1000, is_blocking=True, is_absolute=True, is_enabled=False)
print("Current position: "+ str(ESP32.motor.get_position(axis=1)))

print("Don't worry that it's not going to zero... it actually does, but we are asking for it too fast..")

Current position: 0
Current position: 1000
Current position: 0
Don't worry that it's not going to zero... it actually does, but we are asking for it too fast..


In [5]:
# for moving other motors
ESP32.motor.move_y(steps=1000, speed=1000, is_blocking=True, is_absolute=True, is_enabled=True)
ESP32.motor.move_z(steps=1000, speed=1000, is_blocking=True, is_absolute=True, is_enabled=True)
ESP32.motor.move_t(steps=1000, speed=1000, is_blocking=True, is_absolute=True, is_enabled=True)

''

In [6]:
# moving multiple motors at different speed
ESP32.motor.move_xyz(steps=(1000,160,330), speed=(1000,100,10000), is_blocking=False, is_absolute=False, is_enabled=False)   

''

In [9]:
# moving a motor forever and then stop it
ESP32.motor.move_forever(speed=(100,0,0), is_stop=False)
time.sleep(0.5)
ESP32.motor.move_forever(speed=(100,0,0), is_stop=True)

2022-10-22 23:42:34 DEBUG [UC2Client] Casting json string from serial to Python dict failed
2022-10-22 23:42:35 DEBUG [UC2Client] Casting json string from serial to Python dict failed


''

In [13]:
ESP32.motor.set_motor_maxSpeed(axis=0, maxSpeed=10000)
ESP32.motor.set_motor_currentPosition(axis=0, currentPosition=10000)
ESP32.motor.set_motor_acceleration(axis=0, acceleration=10000)
ESP32.motor.set_motor_enable(is_enable=1)
ESP32.motor.set_direction(axis=1, sign=1, timeout=1)
position = ESP32.motor.get_position(axis=1, timeout=1)
print(position)
ESP32.motor.set_position(axis=1, position=0, timeout=1)

3111


{'return': 1}

# ESP32 State

In [16]:
# test state
_state = ESP32.state.get_state()
print(_state)
ESP32.state.set_state(debug=False)
_mode = ESP32.state.isControllerMode()
print(_mode)
ESP32.state.espRestart()
time.sleep(5)
ESP32.state.setControllerMode(isController=True)
_busy = ESP32.state.isBusy()
print(_busy)


{'identifier_name': 'UC2_Feather', 'identifier_id': 'V1.2', 'identifier_date': 'Sep  3 202205:33:18', 'identifier_author': 'BD', 'IDENTIFIER_NAME': ''}
0


2022-10-22 23:47:05 DEBUG [UC2Client] Casting json string from serial to Python dict failed


False


# LED Matrix

The LED matrix is connected to the "HOLD" pin and can be controlled through the PYTHON interface too

In [17]:
# first define the number of LEDs
ESP32.led.setLEDArrayConfig(ledArrPin=4, ledArrNum=25)

# set all LEDs to a certain RGB value
ESP32.led.send_LEDMatrix_full(intensity=(255, 0, 0), timeout=1)

2022-10-22 23:47:47 DEBUG [UC2Client] Setting LED Pattern (full): (255, 0, 0)


{'return': 1, 'LEDArrMode': 'full'}

In [18]:
# set a single LED to a certain RGB value
ESP32.led.send_LEDMatrix_single(indexled=0, intensity=(0, 255, 0), timeout=1)

2022-10-22 23:48:10 DEBUG [UC2Client] Setting LED PAttern: 0 - (0, 255, 0)


{'return': 1, 'LEDArrMode': 'single'}

In [ ]:
# set a special LED pattern to a certain RGB value (e.g. "top", "bottom", "left", "right")
ESP32.send_LEDMatrix_special(pattern="left", intensity = (255,255,255),timeout=1)

In [19]:
# set a special LED pattern to a certain RGB value (e.g. "top", "bottom", "left", "right")
ESP32.led.send_LEDMatrix_special(pattern="left", intensity = (255,255,255),timeout=1)


2022-10-22 23:48:53 DEBUG [UC2Client] Setting LED Pattern (full): (255, 255, 255)


{'return': 1, 'LEDArrMode': 'left'}

In [20]:
# set a funny pattern
import numpy as np

Nx=8
Ny=8
led_pattern = np.abs(np.int8(np.random.randn(3,Nx*Ny)*255))
ESP32.led.send_LEDMatrix_array(led_pattern=led_pattern, timeout=1)

2022-10-22 23:49:38 DEBUG [UC2Client] Setting LED Pattern (array) 


{'return': 1, 'LEDArrMode': 'array'}

# SLM

In [ ]:
ESP32.send_SLM_circle(posX=10, posY=20, radius=30, color=10000)

In [ ]:
ESP32.send_SLM_clear()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


ESP32.send_SLM_clear()

import time 

time.sleep(1)

Nx=30
Ny=30
image = np.random.randint(0,255,(Nx,Ny))
startX = 0
startY = 0



ESP32.send_SLM_image(image, startX, startY, timeout=3)

'''
endX = startX+image.shape[0]
endY = startY+image.shape[1]
path = '/slm_act'
payload = {
    "task": "/act_slm",
    "color": image[:].flatten().tolist(),
    "startX":startX,
    "startY":startY,
    "endX":endX,
    "endY":endY, 
    "slmMode": "image"
}

payload
'''
plt.imshow(image), plt.show()






# Galvos

This is coming soon

# Lasers

This is coming soon

In [14]:


%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

@widgets.interact(controlLasers=(0, 1000))     
def controlLasers(val=557):
    print(val)
    ESP32.laser.set_laser(channel=1, value=val, despeckleAmplitude=0.5, despecklePeriod=10, timeout=20, is_blocking = True)
    ESP32.laser.set_laser(channel=2, value=val, despeckleAmplitude=0.5, despecklePeriod=10, timeout=20, is_blocking = True)
    ESP32.laser.set_laser(channel=3, value=val, despeckleAmplitude=0.5, despecklePeriod=10, timeout=20, is_blocking = True)



interactive(children=(IntSlider(value=557, description='val', max=1671, min=-557), Output()), _dom_classes=('w…

# LEDs

This is coming soon

# PID controller 

Create a feedback loop for constant pressure